clone coded from : https://www.kaggle.com/code/taroz1461/carrier-smoothing-robust-wls-kalman-smoother/notebook

In [ ]:
!pip install pymap3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import matplotlib.pyplot as plt
import glob as gl
import scipy.optimize
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance

CLIGHT = 299_792_458
RE_WGS84 = 6_378_137
OMGE = 7.2921151467e-5

In [ ]:
def ssatellite_sellection(df, column):
  idx = df[column].notnull()
  idx &= df['CarrierErrorHz'] < 2.0e6
  idx &= df['SvElevationDegrees'] > 10.0
  idx &= df['Cn0DbHz'] > 15.0
  idx &= df['MultipathIndicator'] == 0

  return df[idx]

In [ ]:
def los_vector(xusr, xsat):
  u = xsat - xusr
  rng = np.linalg.norm(u, axis=1).reshape(-1, 1)
  u /= rng
  return u, rng.reshape(-1)

In [ ]:
def jac_pr_residuals(x, xsat,  pr, W):
  u, _ = los_vector(x[:3], xsat)
  J = np.hstack([-u, np.ones([len(pr), 1])])

  return W @ J

In [ ]:
def pr_residuals(x, xsat, pr, W):
  u, rng = los_vector(x[:3], xsat)
  rng += OMGE * (xsat[:, 0] * x[1] - xsat[:, 1] * x[0]) / CLIGHT
  residuals = rng - (pr - x[3])

  return residuals @ W

In [ ]:
def jac_prr_residuals(v, vsat, prr, x, xsat, W):
  u, _ = los_vector(x[:3], xsat)
  J = np.hstack([-u, np.ones([len(prr), 1])])

  return W @ J

In [ ]:
def prr_residuals(v, vsat, prr, x, xsat, W):
  u, rng = los_vector(x[:3], xsat)
  rate = np.sum((vsat-v[:3])*u, axis=1)\
        + OMGE / CLIGHT * (vsat[:, 1] * x[0] + xsat[:, 1] * v[0]\
                           - vsat[:, 0] * x[1] - xsat[:, 0] * v[1])
  residuals = rate - (prr - v[3])

  return residuals @ W  

In [ ]:
def carrier_smoothing(gnss_df):
  carr_th = 1.5
  pr_th = 20.0

  prsmooth   = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)
  for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):
    df = df.replace(
        {'AccumulatedDeltaRangeMeters': {0: np.nan}})

    drng1 = df['AccumulatedDeltaRangeMeters'].diff()\
            - df['PseudorangeRateMetersPerSecond']
    drng2 = df['RawPseudorangeMeters'].diff()\
            - df['PseudorangeRateMetersPerSecond']

    slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0
    slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0
    slip3 = np.fabs(drng1.to_numpy()) > carr_th
    slip4 = np.fabs(drng2.to_numpy()) > pr_th

    idx_slip = slip1 | slip2 | slip3 | slip4
    idx_slip[0] = True

    df['group_slip'] = np.cumsum(idx_slip)

    df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']
    meandpc = df.groupby('group_slip')['dpc'].mean()
    df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean'))

    idx = (gnss_df['Svid'] == svid_sigtype[0])\
         & (gnss_df['SignalType'] == svid_sigtype[1])

    prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']
    
  idx_nan = np.isnan(prsmooth)
  prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]
  gnss_df['pr_smooth'] = prsmooth

  return gnss_df

In [ ]:
# carrier_smoothing replace function
d = pd.DataFrame(columns=['A'])
d['A'] = np.zeros(10)
d['A'] = d['A'].astype(int)
d = d.replace(
    {'A': {0:np.nan}}
)
d

,A
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [ ]:
d = pd.DataFrame(columns=['A'])
d['A'] = np.arange(10)
slip = (d['A'].to_numpy() & 2)
slip

array([0, 0, 2, 2, 0, 0, 2, 2, 0, 0])

In [ ]:
def vincenty_distance(llh1, llh2):
  d, az = np.array(pmv.vdist(llh1[:, 0], llh1[:, 1], 
                             llh2[:, 0], llh2[:, 1]))
  
  return d

def calc_score(llh, llh_gt):
  d = vincenty_distance(llh, llh_gt)
  score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])

  return score

In [ ]:
def point_positioning(gnss_df):
  CarrierFrequencyHzRef = gnss_df.groupby(
      ['Svid', 'SignalType'])['CarrierFrequencyHz'].menian()
  gnss_df = gnss_df.merge(CarrierFrequencyHzRef,
                          how='left',
                          on=['Svid', 'SignalType'],
                          suffixes=('', 'Ref'))
  gnss_df['CarrierErrorHz'] = np.abs(
      (gnss_df['CarrierFrequencyHz'] - gnss_df['CarrierFrequencyHzRef'])
  )

  gnss_df = carrier_smoothing(gnss_df)

  utcTimeMillis = gnss_df['utcTimeMillis'].unique()
  nepoch = len(utcTimeMillis)
  x0 = np.zeros(4)
  v0 = np.zeros(4)
  x_wls = np.full([nepoch, 3], np.nan)
  v_wls = np.full([nepoch, 3], np.nan)
  cov_x = np.full([nepoch, 3, 3], np.nan)
  cov_v = np.full([nepoch, 3, 3], np.nan)

  for i, (t_utc, df) in enumerate(tqdm(gnss_df.groupby('utcTimeMillis'),
                                       total=nepoch)):
    df_pr = ssatellite_sellection(df, 'pr_smooth')
    df_prr = ssatellite_sellection(df, 'PseudorangeRateMetersPerSecond')

    pr = (df_pr['pr_smooth'] + df_pr['SvClockBiasMeters']\
          - df_pr['IsrbMeters'] - df_pr['IonosphericDelayMeters']\
          - df_pr['TroposphericDelayMeters']).to_numpy()
    
    prr = (df_prr['PseudorangeRateMetersPerSecond']\
           + df_prr['SvClockDriftMetersPerSecond']).to_numpy()

    x_sat_pr = df_pr[[
               'SvPositionXEcefMeters', 
               'SvPositionYEcefMeters',
               'SvPositionZEcefMeters']].to_numpy()

    x_sat_prr = df_prr[[
                        'SvPositionXEcefMeters', 
                        'SvPositionYEcefMeters',
                        'SvPositionZEcefMeters']].to_numpy()

    vsat = df_prr[[
                   'SvVelocityXEcefMetersPerSecond', 
                   'SvVelocityYEcefMetersPerSecond',
                   'SvVelocityZEcefMetersPerSecond']].to_numpy()

    Wx = np.diag(1 / df_pr['RawPseudorangeUncertaintyMeters'].to_numpy())
    Wv = np.diag(1 / df_prr['PseudorangeRateUncertaintyMetersPerSecond'].to_numpy())

    if len(df_pr) >= 4:
      if np.all(x0 == 0):
        opt = scipy.optimize.least_squares(
            pr_residuals, x0, jac_pr_residuals, args=(x_sat_pr, pr, Wx)
        )
        x0 = opt.x

      opt = scipy.optimize.least_squares(
          pr_residuals, x0, jac_pr_residuals,
          args=(x_sat_pr, pr, Wx), loss='soft_l1'
      )
      if opt.status < 1 or opt.status == 2:
        print(f'i = {i} position lsq status = {opt.status}')
      else:
        cov = np.linalg.inv(opt.jac.T @ Wx @ opt.jac)
        cov_x[i, :, :] = cov[:3, :3]
        x_wls[i, :] = opt.x[:3]
        x0 = opt.x
    if len(df_prr) >= 4:
      if np.all(v0 == 0):
        opt = scipy.optimize.least_squares(
            prr_residuals, v0, jac_prr_residuals,
            args=(vsat, prr, x0, x_sat_prr, Wv)
        )
        v0 = opt.x
      
      opt = scipy.optimize.least_squares(
          prr_residuals, v0, jac_prr_residuals,
          args=(vsat, prr, x0, x_sat_prr, Wv), loss='soft_l1'
      )
      if opt.status < 1:
        print(f'i = {i} velocity lsq status = {opt.status}')
      else:
        cov = np.linalg.inv(opt.jac.T @ Wv @ opt.jac)
        cov_v[i, :, :] = cov[:3, :3]
        v_wls[i, :] = opt.x[:3]
        v0 = opt.x
  return utcTimeMillis, x_wls, v_wls, cov_x, cov_v

In [ ]:
a = np.arange(0, 4).reshape(2, 2)
b = np.arange(0, 4).reshape(2, 2)
a, b, a@b

(array([[0, 1],
        [2, 3]]), array([[0, 1],
        [2, 3]]), array([[ 2,  3],
        [ 6, 11]]))

In [ ]:
def exclude_interpolate_outlier(x_wls, v_wls, cov_x, cov_v):
  v_up_th = 2.6
  height_th = 200.0
  v_out_sigma = 3.0
  x_out_sigma = 30.0

  x_llh = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], 
                                    x_wls[:, 2])).T
  x_llh_mean = np.nanmean(x_llh, axis=0)
  v_enu = np.array(pm.ecef2enuv(
      v_wls[:, 0], v_wls[:, 1], v_wls[:, 2], 
      x_llh_mean[0], x_llh_mean[1]
  )).T

  idx_v_out = np.abs(v_enu[:, 2]) > v_up_th
  idx_v_out |= np.isnan(v_enu[:, 2])
  v_wls[idx_v_out, :] = np.nan
  cov_v[idx_v_out] = v_out_sigma**2 * np.eye(3)
  print(f'Number of velocity outliers {np.count_nonzero(idx_v_out)}')

  hmedian = np.nanmedian(x_llh[:, 2])
  idx_x_out = np.abs(x_llh[:, 2] - hmedian) > height_th
  idx_x_out |= np.isnan(x_llh[:, 2])
  x_wls[idx_x_out, :] = np.nan
  cov_x[idx_x_out] = x_out_sigma**2 * np.eye(3)
  print(f'Number of position outliers {np.count_nonzero(idx_x_out)}')

  x_df = pd.DataFrame({'x': x_wls[:, 0], 
                       'y': x_wls[:, 1], 
                       'z': x_wls[:, 2]})
  x_df = x_df.interpolate(limit_area='outside',
                          limit_direction='both')
  
  v_df = pd.DataFrame({'x': v_wls[:, 0], 
                       'y': v_wls[:, 1], 
                       'z': v_wls[:, 2]})
  v_df = v_df.interpolate(limit_area='outside',
                          limit_direction='both')
  v_df = v_df.interpolate('spline', order=3)

  return x_df.to_numpy(), v_df.to_numpy(), cov_x, cov_v

In [ ]:
def Kalman_filter(zs, us, cov_sz, cov_us, phone):
  sigma_mahalanobis = 30.0

  n, dim_x = zs.shape
  F = np.eye(3)
  H = np.eye(3)

  x = zs[0, :3].T
  P = 5.0**2 * np.eye(3)
  I = np.eye(dim_x)

  x_kf = np.zeros([n, dim_x])
  P_kf = np.zeros([n, dim_x, dim_x])

  for i, (u, z) in enumerate(zip(us, zs)):
    if i == 0:
      x_kf[i] = x.T
      P_kf[i] = P
      continue
    
    Q = cov_us[i]

    x = F @ x + u.T
    P = (F @ P) @ F.T + Q

    d = distance.mahalanobis(x, H @ x, np.linalg.inv(P))

    if d < sigma_mahalanobis:
      R = cov_sz[i]
      y = z.T - H @ x
      S = (H @ P) @ H.T + R
      K = (P @ H.T) @ np.linalg.inv(S)
      x = x + K @ y
      P = (I - (K @ H)) @ P
    else:
      P += 10**2*Q
      x_kf[i] = x.T
      P_kf[i] = P

  return x_kf, P_kf

In [1]:
def Kalman_smoothing(x_wls, v_wls, cov_x, cov_v, phone):
  n, dim_x = x_wls.shape

  if phone == 'XiaomiMi8':
    v_wls = np.v_stack(
        [(v_wls[:-1, :] + v_wls[1:, :])/2, np.zeros([1, 3])]
    )
    cov_v = 1000.0**2 * cov_v

  v = np.vstack(
      [np.zeros([1, 3]), (v_wls[:-1, :] + v_wls[1:, :])/2]
  )
  x_f, P_f = Kalman_filter(
      x_wls, v, cov_x, cov_v, phone
  )

  v = -np.flipud(v_wls)
  v = np.vstack(
      [np.zeros([1, 3]), (v[:-1, :] + v[1:, :])/2]
  )
  cov_xf = np.flip(cov_x, axis=0)
  cov_vf = np.flip(cov_v, axis=0)
  x_b, P_b = Kalman_filter(
      np.flipud(x_wls), v, cov_xf, cov_vf, phone
  )

  x_fb = np.zeros_like(x_f)
  P_fb = np.zeros_like(P_f)
  for (f, b) in zip(range(n), range(n-1, -1, -1)):
    P_fi = np.linalg.inv(P_f[f])
    P_bi = np.linalg.inv(P_b[b])

    P_fb[f] = np.linalg.inv(P_fi + P_bi)
    x_fb[f] = P_fb[f] @ (P_fi @ x_f[f] + P_bi @ x_b[b])
  
  return x_fb, x_f, np.flipud(x_b)
    